### Employment from Statistics Canada
Data downloaded from https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1410038702  
Data from 2006 - ongoing

In [3]:
import pandas as pd 
import matplotlib.pyplot as plt
import datetime
import plotly.graph_objects as go
import numpy as np

In [ ]:
### Download latest table

import requests, zipfile, io
r = requests.get("https://www150.statcan.gc.ca/n1/tbl/csv/14100387-eng.zip")
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(f"../data/raw")

In [213]:
## Read original source Labor Force Survey 
data = pd.read_csv(f'../data/raw/14100387-eng/14100387.csv')

/Users/tamarahuete/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning:

Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.



In [214]:
data.head()

,REF_DATE,GEO,DGUID,Labour force characteristics,Statistics,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2006-03,Canada,2016A000011124,Population,Estimate,Persons,249,thousands,3,v1235063379,1.1.1,25979.9,NaN,NaN,NaN,1
1,2006-03,Canada,2016A000011124,Labour force,Estimate,Persons,249,thousands,3,v1235063380,1.2.1,17124.8,NaN,NaN,NaN,1
2,2006-03,Canada,2016A000011124,Labour force,Standard error of estimate,Persons,249,thousands,3,v1235063381,1.2.2,36.6,NaN,NaN,NaN,1
3,2006-03,Canada,2016A000011124,Labour force,Standard error of year-over-year change,Persons,249,thousands,3,v1235063382,1.2.3,NaN,..,NaN,NaN,1
4,2006-03,Canada,2016A000011124,Employment,Estimate,Persons,249,thousands,3,v1235063383,1.3.1,15943.7,NaN,NaN,NaN,1


In [215]:
### Select only data for Bristish Columbia
data = data[(data.GEO.isin(['British Columbia',
       'Vancouver Island and Coast, British Columbia',
       'Lower Mainland-Southwest, British Columbia',
       'Thompson-Okanagan, British Columbia',
       'Kootenay, British Columbia', 'Cariboo, British Columbia',
       'North Coast and Nechako, British Columbia',
       'Northeast, British Columbia'])) & (data.Statistics == 'Estimate')]

In [216]:
data.GEO.unique()

array(['British Columbia', 'Vancouver Island and Coast, British Columbia',
       'Lower Mainland-Southwest, British Columbia',
       'Thompson-Okanagan, British Columbia',
       'Kootenay, British Columbia', 'Cariboo, British Columbia',
       'North Coast and Nechako, British Columbia',
       'Northeast, British Columbia'], dtype=object)

In [217]:
data['Labour force characteristics'].unique()

array(['Population', 'Labour force', 'Employment', 'Full-time employment',
       'Part-time employment', 'Unemployment', 'Not in labour force',
       'Unemployment rate', 'Participation rate', 'Employment rate'],
      dtype=object)

In [218]:
data.columns

Index(['REF_DATE', 'GEO', 'DGUID', 'Labour force characteristics',
       'Statistics', 'UOM', 'UOM_ID', 'SCALAR_FACTOR', 'SCALAR_ID', 'VECTOR',
       'COORDINATE', 'VALUE', 'STATUS', 'SYMBOL', 'TERMINATED', 'DECIMALS'],
      dtype='object')

In [219]:
## Select only useful columns
data = data[['REF_DATE', 'GEO','Labour force characteristics','UOM','SCALAR_FACTOR','VALUE']]

In [220]:
data.head()

,REF_DATE,GEO,Labour force characteristics,UOM,SCALAR_FACTOR,VALUE
1904,2006-03,British Columbia,Population,Persons,thousands,3422.1
1905,2006-03,British Columbia,Labour force,Persons,thousands,2205.4
1908,2006-03,British Columbia,Employment,Persons,thousands,2092.2
1911,2006-03,British Columbia,Full-time employment,Persons,thousands,1640.2
1914,2006-03,British Columbia,Part-time employment,Persons,thousands,452.0


In [221]:
### Keep only data from 2016 onwards
data = data[pd.DatetimeIndex(data['REF_DATE']).year > 2015]

In [222]:
data.GEO.unique()

array(['British Columbia', 'Vancouver Island and Coast, British Columbia',
       'Lower Mainland-Southwest, British Columbia',
       'Thompson-Okanagan, British Columbia',
       'Kootenay, British Columbia', 'Cariboo, British Columbia',
       'North Coast and Nechako, British Columbia',
       'Northeast, British Columbia'], dtype=object)

In [223]:
## Sneak peak into data
variables = ['Population','Labour force','Employment','Unemployment']

fig,axes = plt.subplots(3,2,figsize = (10,10))
for a,ax in enumerate(axes.flatten()):
    df = data[(data['GEO'] == data.GEO.unique()[a])]
    for variable in variables:
        ax.plot(pd.DatetimeIndex(df[df['Labour force characteristics'] == variable]['REF_DATE']),df[df['Labour force characteristics'] == variable]['VALUE'])
        ax.set_title(f'{data.GEO.unique()[a]}')

In [224]:
## Replace Economic Region names with corresponding Tourism Regions
correspondence = {'British Columbia':'British Columbia',
                  'Vancouver Island and Coast, British Columbia': 'Vancouver Island',
                  'Lower Mainland-Southwest, British Columbia':'Vancouver Coast and Mountains',
                  'Thompson-Okanagan, British Columbia':  'Thompson-Okanagan',
                  'Kootenay, British Columbia': 'Kootenay Rockies', 
                  'Cariboo, British Columbia':'Cariboo Chilcotin Coast',
                  'North Coast and Nechako, British Columbia':'Northern British Columbia',
                  'Northeast, British Columbia':'Northern British Columbia'}

In [225]:
data['Tourism_region'] =data['GEO']

In [226]:
data.replace({'Tourism_region':correspondence}, inplace=True)

In [227]:
data.columns

Index(['REF_DATE', 'GEO', 'Labour force characteristics', 'UOM',
       'SCALAR_FACTOR', 'VALUE', 'Tourism_region'],
      dtype='object')

In [228]:
## Get values in appropriate units
data_count = data[data['Labour force characteristics'].isin(['Population','Labour force','Employment','Unemployment'])]
#data_count = data_count.groupby(['Tourism_region','REF_DATE','Labour force characteristics']).sum().reset_index()
data_count['VALUE']= data_count['VALUE']*1000

/Users/tamarahuete/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [229]:
data_per = data[data['Labour force characteristics'].isin(['Employment rate','Unemployment rate'])]
#data_per = data_per.groupby(['Tourism_region','REF_DATE','Labour force characteristics']).mean().reset_index()

In [230]:
df = data_count.append(data_per)

In [231]:
### Format data with data model for database
df.rename(columns={'REF_DATE':'date',
                   'GEO':'category_2',
                   'Tourism_region':'region',
                   'VALUE':'value',
                   'Labour force characteristics':'indicator_code'}, inplace=True)

In [232]:
df['category_1']=np.nan

In [233]:
df = df[['indicator_code','date','region','category_1','category_2','value']]

In [234]:
## Set indicator names
indicator_names = {'Population':'population_by_economic_region',
                  'Labour force': 'labour_force_by_economic_region',
                  'Employment':'total_employment_by_economic_region',
                  'Unemployment':  'total_unemployment_by_economic_region',
                  'Employment rate':'total_employment_rate_by_economic_region',
                  'Unemployment rate':'total_unemployment_rate_by_economic_region'}

In [235]:
df.replace({'indicator_code':indicator_names}, inplace=True)

In [236]:
### delete 'British Columbia tag'
## Replace Economic Region names with corresponding Tourism Regions
correspondence = {'British Columbia':'British Columbia',
                  'Vancouver Island and Coast, British Columbia': 'Vancouver Island and Coast',
                  'Lower Mainland-Southwest, British Columbia':'Lower Mainland-Southwest',
                  'Thompson-Okanagan, British Columbia':  'Thompson-Okanagan',
                  'Kootenay, British Columbia': 'Kootenay', 
                  'Cariboo, British Columbia':'Cariboo',
                  'North Coast and Nechako, British Columbia':'North Coast and Nechako',
                  'Northeast, British Columbia':'Northeast'}

In [237]:
df.replace({'category_2':correspondence}, inplace=True)

In [240]:
df.sort_values(by=['indicator_code','date']).to_csv(f'../data/processed/Labour_Force_Survey_2016_2021.csv')